## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-28-22-51-58 +0000,nyt,Trump Administration Live Updates: President M...,https://www.nytimes.com/live/2025/07/28/us/tru...
1,2025-07-28-22-51-44 +0000,nyt,2 Children in Sailing Program Die After Barge ...,https://www.nytimes.com/2025/07/28/us/sailboat...
2,2025-07-28-22-47-24 +0000,wapo,Alabama’s body-camera law undermines police ac...,https://www.washingtonpost.com/nation/2025/07/...
3,2025-07-28-22-47-06 +0000,startribune,"DHS: 140,000 Minnesotans could lose health ins...",https://www.startribune.com/dhs-140000-minneso...
4,2025-07-28-22-45-45 +0000,startribune,‘I’m going to murder us all’: Isle Royale camp...,https://www.startribune.com/im-going-to-murder...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2445/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,62
248,deal,26
247,trade,24
118,gaza,16
179,new,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
167,2025-07-28-14-42-00 +0000,wsj,Trump said the U.S. reached a trade deal with ...,https://www.wsj.com/economy/trade/european-uni...,139
103,2025-07-28-18-24-16 +0000,nypost,France presses EU to unload ‘trade bazooka’ on...,https://nypost.com/2025/07/28/business/france-...,139
197,2025-07-28-11-56-36 +0000,nyt,"Europe, Worried by Other Global Crises, Agrees...",https://www.nytimes.com/2025/07/28/world/europ...,129
124,2025-07-28-17-17-00 +0000,wsj,President Trump said he would give Russia’s Vl...,https://www.wsj.com/politics/trump-losing-pati...,122
136,2025-07-28-16-27-53 +0000,nypost,Trump hails cease-fire win after halting trade...,https://nypost.com/2025/07/28/us-news/trump-ha...,121


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
167,139,2025-07-28-14-42-00 +0000,wsj,Trump said the U.S. reached a trade deal with ...,https://www.wsj.com/economy/trade/european-uni...
124,60,2025-07-28-17-17-00 +0000,wsj,President Trump said he would give Russia’s Vl...,https://www.wsj.com/politics/trump-losing-pati...
67,51,2025-07-28-20-32-48 +0000,nypost,"Abandoned NYC airport will land 3,000 new home...",https://nypost.com/2025/07/28/us-news/abandone...
73,46,2025-07-28-20-01-59 +0000,nyt,"In a First, Leading Israeli Rights Groups Accu...",https://www.nytimes.com/2025/07/28/world/middl...
129,42,2025-07-28-16-50-00 +0000,wsj,Business leaders on both sides of the Atlantic...,https://www.wsj.com/economy/trade/companies-we...
164,39,2025-07-28-15-04-00 +0000,wsj,Former Gov. Roy Cooper of North Carolina launc...,https://www.wsj.com/politics/elections/roy-coo...
26,36,2025-07-28-21-40-06 +0000,nypost,Heartbreaking video shows family’s tearful sen...,https://nypost.com/2025/07/28/us-news/family-t...
29,34,2025-07-28-21-36-00 +0000,wsj,"Stock Market Today: S&P, Nasdaq Edge Higher to...",https://www.wsj.com/livecoverage/stock-market-...
176,34,2025-07-28-14-10-37 +0000,wapo,Thailand and Cambodia agree to ceasefire after...,https://www.washingtonpost.com/world/2025/07/2...
0,28,2025-07-28-22-51-58 +0000,nyt,Trump Administration Live Updates: President M...,https://www.nytimes.com/live/2025/07/28/us/tru...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
